In [ ]:
player_name = input("What is the name of the player you want to search for? ")
platform = input("What platform is the player on? ")
type = input("What is the type of the player? ")

In [ ]:
processed_player_name = player_name.replace(" ", "+")
processed_platform = platform.lower()
processed_type = type.lower()

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os

query = f"https://www.bcci.tv/search?platform={processed_platform}&type={processed_type}&term={processed_player_name}&content_type=all"

if not processed_player_name or not processed_platform or not processed_type:
    print("Invalid input")
    exit()
else:
    try:
        response = requests.get(query)
        response.raise_for_status()
    except requests.RequestException as e:
        print(e)

    soup = BeautifulSoup(response.text, 'html.parser')
    data = []

    player_data = soup.find('div', class_='lv pc video-section-append-video-here')
    if player_data:
        player_card = player_data.find_all(
            'div', class_='slick-card m-0 lv-bg hoverVideoPlayNow')
        for card in player_card:
            header_content = card.find('div', class_='bottom')
            title_holder = None
            title = ""
            date = ""
            views = ""
            if header_content.find('div', class_='text-detail br-b'):
                title_holder = header_content.find('div', class_='text-detail br-b')
                date_holder = header_content.find('div', class_='text-detail br-b')
                date =  date_holder.find('span').text
                views_holder = header_content.find(
                    'div', class_='tour-overlay-details')
                list = views_holder.find('ul')
                views = list.find_all('li')[0]
                views = views.find('span', class_="me-3").text
                views = views.replace("&nbsp;", "").replace(
                    "views", "").replace("\n", "").replace("\xa0", "").strip()
                if "k" in views:
                    views = views.replace("k", "")
                    views = float(views) * 1000
                elif "m" in views:
                    views = views.replace("m", "")
                    views = float(views) * 1000000
            elif header_content.find('div', class_='text-detail pb-0'):
                title_holder = header_content.find('div', class_='text-detail pb-0')
                date_holder = header_content.find(
                    'div', class_='tour-overlay-details')
                ul = date_holder.find('ul')
                date_ = ul.find_all('li')[0]
                date = date_.find('span').text
                views = "-"
            if title_holder:
                title = title_holder.find('p').text

            data.append({
                "title": title,
                "date": date,
                "views": views,
                "platform": processed_platform,
                "type": processed_type,
                "player_name": player_name,
                "sport": "Cricket"
            })
        if data:
            df = pd.DataFrame(data)
            header = ['title', 'date', 'views', 'platform', 'type', 'player_name']
            if os.path.isfile('Sports.csv') and os.path.getsize('Sports.csv') > 0:
                df.to_csv('Sports.csv', mode='a', header=False, index=False)
            else:
                df.to_csv('Sports.csv', mode='a', header=header, index=False)
    else:
        print("Player not found")